In [2]:
import requests
import json
from bs4 import BeautifulSoup

In [3]:
data = []
with open('result.json') as f:
        data = json.load(f)

In [ ]:
url = "https://www.retailmenot.com/coupons/food"

In [ ]:
response = requests.get(url)
if response.status_code != 200:
    print("Unable to retrieve HTML")

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
deals_container = soup.find('div', attrs={'class', 'tile__group'})

In [ ]:
deals = deals_container.find_all('article', attrs={'class', 'tile'})

In [ ]:
def next_space(idx, inp_str):
    for i in range(idx, len(inp_str)):
        if inp_str[i] == ' ':
            return i
    return len(inp_str)
def prev_space(idx, inp_str):
    for i in range(idx, -1, -1):
        if inp_str[i] == ' ':
            return i
    return 0

In [17]:
def no_punc(description):
    return description.replace(".","").replace(",","").replace("?","").replace("!","")

In [19]:
def estimate_deal(description):
    deal = {}
    description = nopunc(description)
    if '$' in description:
        if "off" in description.lower() or "credit" in description.lower():
            deal["type"] = "D" #stands for deductible
            starting_idx = description.find('$')+1
            deal["amt"] = float(description[starting_idx:next_space(starting_idx, description)])
        else:
            deal["type"] = "R" #stands for reduced cost
            starting_idx = description.find('$')+1
            deal["amt"] = float(description[starting_idx:next_space(starting_idx, description)])
    elif 'free' in description.lower():
        deal["type"] = "R" #stands for cost
        deal["amt"] = 0
    elif '%' in description:
        deal["type"] = "P" #stands for percent
        starting_idx = description.find('%')
        deal["amt"] = float(description[prev_space(starting_idx, description):starting_idx])
    else:
        deal["type"] = "G" #stands for general deal
        deal["amt"] = 0
    return deal

In [20]:
all_deals = []
for deal in deals:
    deals_obj = {}
    inner_container = deal.find('a')
    if not inner_container.has_attr("class") or not inner_container["class"][0] == "content":
        deals_obj["link"] = "https://www.myunidays.com" + inner_container['href']
        merchant_details = inner_container.find('div', attrs={'class': 'tile__inner'})
        description = merchant_details.find('p', attrs={'class': 'tile__discount'})
        deals_obj["description"] = description.text
        merchant = merchant_details.find('div', attrs={'class': 'tile__logo'}).find('img', attrs={'class': 'c-perk-logo'})["alt"]
        deals_obj["merchant"] = merchant
        deals_obj["category"] = "technology"
        deals_obj["deal_estimate"] = estimate_deal(description.text)
        all_deals.append(deals_obj)

NameError: name 'deals' is not defined

In [ ]:
all_deals

In [ ]:
new_data = data + all_deals

In [ ]:
with open('result.json', 'w') as fp:
    json.dump(new_data, fp)